In [1]:
import torch
import matplotlib.pyplot as plt
import torch_dct as dct
from lib.tools import *
from lib.decomposition import *
from lib.jpeg import *
from lib.sketchml import *
from lib.regression import *
import os
import pandas as pd

In [2]:
# tensor_dir = "../experiments_fotball/tensors/"
layer =60
tensor_dir = "./test_tensors/coco_tensor_yolov3/layer_"+str(layer)+"/"
output_dir = "./measurements/coco_test/"


test_tensors = os.listdir(tensor_dir)

In [3]:
# def sort_fun(tensor_name):
#     return int(tensor_name.split("_")[2].split(".")[0])

# test_tensors.sort(key=sort_fun)

In [4]:
## Measurements
split_layer = []

tensor_rank=[]
tensor_sparsity=[]
tensor_regularity=[]
tensor_pictoriality=[]
tensor_decomposability=[]

jpeg_ratio=[]
jpeg_snr=[]
jpeg_mse=[]
decomposition_ratio=[]
decomposition_snr=[]
decomposition_mse=[]
regression_ratio=[]
regression_snr=[]
regression_mse=[]
sketchml_ratio=[]
sketchml_snr=[]
sketchml_mse=[]

In [5]:
for tensor_name in test_tensors[0:20]:
    print("************* Testing tensor: "+tensor_name+"***************")
    split_layer.append(layer)
    tensor = torch.load(tensor_dir+tensor_name)
    # tensor = tensor * (abs(tensor)>0.001)
    tensor = tensor[0]

    # tensor_rank.append(calculate_cp_rank(tensor.numpy(),max_rank=10,tol=1e-4))
    tensor_rank.append(calculate_slice_avg_rank(tensor))
    tensor_sparsity.append(calculate_sparsity(tensor.numpy()))
    tensor_pictoriality.append(get_tensor_pictoriality(tensor))
    tensor_regularity.append(get_tensor_regularity(tensor))
    tensor_decomposability.append(get_tensor_decomposability(tensor))

    print("Rank: %f, Sparsity: %f, Picoriality: %f, Regularity: %f, Decomposability: %f" % (tensor_rank[-1],tensor_sparsity[-1],tensor_pictoriality[-1],tensor_regularity[-1],tensor_decomposability[-1]))

    ## Jpeg compression
    print("Doing JPEG compression")
    normalize_base, scale,zero_point, encoded_data, compressed_size_jpeg = compressor_jpeg_gray(tensor, 100)
    reconstructed_tensor_jpeg = decompressor_jpeg_gray(tensor.shape,normalize_base, scale,zero_point, encoded_data )
    ratio_jpeg = tensor.numel()*4/compressed_size_jpeg
    snr_jpeg = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_jpeg.numpy())
    mse_jpeg = calculate_mse(tensor.numpy(), reconstructed_tensor_jpeg.numpy())
    jpeg_ratio.append(ratio_jpeg)
    jpeg_snr.append(snr_jpeg)
    jpeg_mse.append(mse_jpeg)
    print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_jpeg,snr_jpeg,mse_jpeg))

    print("Doing Decomposition compression")
    # factors, compressed_size_decomposition = compressor_decomposition(tensor, tensor_rank[-1])
    # reconstructed_tensor_decomposition = decompressor_decomposition(factors)
    factors, compressed_size_decomposition = compressor_decomposition_slice(tensor)
    reconstructed_tensor_decomposition = decompressor_decomposition_slice(tensor.shape,factors)
    snr_decomposition = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_decomposition.numpy())
    mse_decomposition = calculate_mse(tensor.numpy(), reconstructed_tensor_decomposition.numpy())
    ratio_decomposition= tensor.numel()*4/compressed_size_decomposition
    decomposition_ratio.append(ratio_decomposition)
    decomposition_snr.append(snr_decomposition)
    decomposition_mse.append(mse_decomposition)
    print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_decomposition,snr_decomposition,mse_decomposition))

    print("Doing Regression compression")
    factors, x_pos, x_neg, compressed_size_regression = compressor_regression(tensor, 3)
    reconstructed_tensor_regression = decompressor_regression(tensor.shape, factors, x_pos,x_neg )
    snr_regression = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_regression.numpy())
    mse_regression = calculate_mse(tensor.numpy(), reconstructed_tensor_regression.numpy())
    ratio_regression = tensor.numel()*4 / compressed_size_regression
    regression_ratio.append(ratio_regression)
    regression_snr.append(snr_regression)
    regression_mse.append(mse_regression)
    print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_regression,snr_regression,mse_regression))

    print("Doing SketchML compression")
    minmaxsketch,bucket_means, keys, compressed_size_sketchml = compressor_sketchml(tensor,256,0.3,3)
    reconstructed_tensor_sketchml = decompressor_sketchml(tensor.shape, minmaxsketch,bucket_means, keys)
    snr_sketchml = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_sketchml.numpy())
    mse_sketchml = calculate_mse(tensor.numpy(), reconstructed_tensor_sketchml.numpy())
    ratio_sketchml = tensor.numel() *4 / compressed_size_sketchml
    sketchml_ratio.append(ratio_sketchml)
    sketchml_snr.append(snr_sketchml)
    sketchml_mse.append(mse_sketchml)
    print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_sketchml,snr_sketchml,mse_sketchml))


************* Testing tensor: 20.pt***************
Rank: 25.996094, Sparsity: 0.000000, Picoriality: 0.594862, Regularity: 0.395257, Decomposability: -0.999700
Doing JPEG compression
Ratio: 7.850659, SNR: 20.019307, MSE: 0.000316
Doing Decomposition compression
Ratio: 2.000301, SNR: 133.496552, MSE: 0.000000
Doing Regression compression
Ratio: 14.939227, SNR: 1.721574, MSE: 0.021344
Doing SketchML compression
Ratio: 4.961432, SNR: 0.062462, MSE: 0.031275
************* Testing tensor: 18.pt***************
Rank: 25.996094, Sparsity: 0.000000, Picoriality: 0.530492, Regularity: 0.381783, Decomposability: -0.999700
Doing JPEG compression
Ratio: 6.328442, SNR: 20.629234, MSE: 0.000320
Doing Decomposition compression
Ratio: 2.000301, SNR: 134.049730, MSE: 0.000000
Doing Regression compression
Ratio: 14.939227, SNR: 1.755466, MSE: 0.024710
Doing SketchML compression
Ratio: 4.961432, SNR: 0.030944, MSE: 0.036756
************* Testing tensor: 8.pt***************
Rank: 25.996094, Sparsity: 0.000

KeyboardInterrupt: 

In [6]:
dict = {'split_layer':split_layer,
        'rank': tensor_rank, 
        'sparsity': tensor_sparsity, 
        'regularity': tensor_regularity,
        'pictoriality': tensor_pictoriality,
        'decomposability':tensor_decomposability,
        'jpeg_ratio':jpeg_ratio,
        'jpeg_snr':jpeg_snr,
        'jpeg_mse':jpeg_mse,
        'decomposition_ratio':decomposition_ratio,
        'decomposition_snr':decomposition_snr,
        'decomposition_mse':decomposition_mse,
        'regression_ratio':regression_ratio,
        'regression_snr':regression_snr,
        'regression_mse':regression_mse,
        'sketchml_ratio':sketchml_ratio,
        'sketchml_snr':sketchml_snr,
        'sketchml_mse':sketchml_mse
        } 
   
df = pd.DataFrame(dict)

In [7]:
df.to_csv(output_dir+"layer"+str(layer)+".csv",index=False, header=True)

In [ ]:
df.head()

,split_layer,rank,sparsity,regularity,pictoriality,decomposability,jpeg_ratio,jpeg_snr,jpeg_mse,decomposition_ratio,decomposition_snr,decomposition_mse,regression_ratio,regression_snr,regression_mse,sketchml_ratio,sketchml_snr,sketchml_mse
0,20,13.0,0.0,0.511939,0.909452,0.0,4.896749,24.492781,0.002022,4.0,133.686924,2.434139e-14,14.939227,4.817267,0.187634,4.847914,0.463375,0.511328
1,20,13.0,0.0,0.517800,0.887979,0.0,5.172954,21.596024,0.005342,4.0,134.328423,2.847434e-14,14.939227,4.740386,0.258974,4.851312,0.013533,0.769028
2,20,13.0,0.0,0.472356,0.842249,0.0,5.069307,26.324272,0.001585,4.0,134.053116,2.674653e-14,14.939227,4.726471,0.229051,4.849884,-0.158891,0.705454
3,20,13.0,0.0,0.488877,0.899398,0.0,4.300543,24.206805,0.002324,4.0,133.739176,2.587807e-14,14.939227,4.815726,0.201965,4.852604,0.403193,0.557866
4,20,13.0,0.0,0.481023,0.878923,0.0,4.648856,21.166656,0.006447,4.0,134.389238,3.069645e-14,14.939227,3.732624,0.357067,4.853625,0.121171,0.820153
